In [ ]:
from bot_studio import *

In [ ]:
import os
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time

from selenium import webdriver

def start_browser():
    env_key = "YTCRAWLER_BROWSER"
    path = os.getcwd()

    browser = os.environ.get(env_key)
    if(browser == None):
        print("No browser defined: ")
        browser = input()
        browser = os.environ[env_key] = browser
    else:
        print("Defined browser: {}", browser)

    if browser == "chrome":
        from webdriver_manager.chrome import ChromeDriverManager
        chrome_options = webdriver.ChromeOptions()
        chrome_options.add_argument("--mute-audio")
        
        browser = webdriver.Chrome(ChromeDriverManager().install(),options=chrome_options)
    elif browser == "firefox":
        options = webdriver.FirefoxOptions()
        options.set_preference("media.autoplay.default", 0)  # Allow video auto-play
        options.set_preference("media.volume_scale", "0.0")  # Mute sound

        browser = webdriver.Firefox(options=options)
        browser.install_addon(path + "/extensions/firefox/ublock_origin.xpi")
    else:
        os.environ.pop(env_key)
        raise ValueError("Unknown browser: {}".format(browser))

    browser.get("https://www.youtube.com")
    time.sleep(5)
    # Remove consent popup
    buttons = browser.find_element(By.LINK_TEXT,'I AGREE')
    buttons.click()
    
    return browser


browser = start_browser()

In [ ]:
FIRST_VIDEO_POSITION = 0
NEXT_VIDEO_POSITION = 0
WATCH_TIME_VIDEOS = 5
MAX_VIDEOS = 30


def click_on_next_video(browser, next_video_position=0):
    videos = browser.find_elements(By.XPATH, "//a[@id='thumbnail'][@href]")
    next_video = videos[next_video_position]
    href = next_video.get_property('href')
    next_video.click()
    time.sleep(1)
    browser.refresh()
    return href

cpt=0
next_video = FIRST_VIDEO_POSITION
watched_videos = []

while cpt<MAX_VIDEOS:
    time.sleep(2)
    cpt+=1
    watched_videos.append(click_on_next_video(browser, next_video))
    next_video = NEXT_VIDEO_POSITION
    
    if len(browser.find_elements(By.XPATH, "//button[@title='Play (k)']"))>0:
        browser.find_element(By.XPATH, "//button[@title='Play (k)']").click()
    time.sleep(WATCH_TIME_VIDEOS)
    
browser.close()